In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
ls data/sports_scores/

astros_1A10.csv*   Rice_10H80.csv*    Tex_15E40.csv*
dynamo_10H30.csv*  rockets_1A10.csv*  UH_10H50.csv*


In [3]:
ls data/crime_beat/

crime_10H30_10_17.csv*  crime_10H80_10_17.csv*  crime_1A10_10_17.csv*
crime_10H50_10_17.csv*  crime_15E40_10_17.csv*


In [4]:
games = 'data/sports_scores/UH_10H50.csv'

crime = 'data/crime_beat/crime_10H50_10_17.csv'

# scores df

In [5]:
df = pd.read_csv(games,index_col='Date').sort_index(ascending=True)

## win column
df['win'] =  np.where(df['Pts'] > df['Opp'],1,0)

# only oposing name and win boolean
df = df[['Opponent','win']]

# show

df.head()

,Opponent,win
Date,,
2010-09-04,Texas State,1
2010-09-10,UTEP,1
2010-09-25,Tulane,1
2010-10-09,Mississippi State,0
2010-11-05,UCF,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49 entries, 2010-09-04 to 2017-11-24
Data columns (total 2 columns):
Opponent    49 non-null object
win         49 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


# Crime df

In [7]:
crime_df = pd.read_csv(crime).sort_index(ascending=True)

crime_df['Date_Hour'] = pd.to_datetime(crime_df['Date_Hour'])
crime_df['Date_Hour'] = crime_df['Date_Hour'].dt.date

crime_df = crime_df.set_index('Date_Hour')


# selectd columns
crime_df = crime_df[['OffenseType']]

# count Offense type given date
date_crimes = crime_df.groupby(crime_df.index)['OffenseType',].count()

## show
date_crimes.head()

,OffenseType
Date_Hour,
2010-01-01,5
2010-01-03,1
2010-01-04,6
2010-01-05,1
2010-01-06,2


In [8]:
#crime_df = pd.get_dummies(crime_df,drop_first=True)

In [9]:
#crime_df.info()

# Days Df
- create a df with everyday timestamp from Jan 2010 to Dec 2017

In [10]:
days = pd.date_range(start='01/01/2010', end='12/31/2017')
days = pd.DataFrame(days)

days.columns = ['days']
days = days.set_index('days').sort_index(ascending=True)
days.head()

""
days
2010-01-01
2010-01-02
2010-01-03
2010-01-04
2010-01-05


In [11]:
365 * 8

2920

In [12]:
len(days)/365

8.005479452054795

In [13]:
days.tail()

""
days
2017-12-27
2017-12-28
2017-12-29
2017-12-30
2017-12-31


# Merge days df with crimes_df

In [14]:
calendar_crimes = pd.merge(days,date_crimes,  left_index=True, right_index=True, how='left')
calendar_crimes.head()

,OffenseType
days,
2010-01-01,5.0
2010-01-02,NaN
2010-01-03,1.0
2010-01-04,6.0
2010-01-05,1.0


In [15]:
calendar_crimes.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 1 columns):
OffenseType    2714 non-null float64
dtypes: float64(1)
memory usage: 125.7 KB


In [16]:
calendar_crimes.OffenseType.value_counts(dropna=False)

 2.0     707
 3.0     598
 1.0     551
 4.0     416
 5.0     224
NaN      208
 6.0     129
 7.0      46
 8.0      28
 9.0       7
 12.0      3
 10.0      3
 14.0      1
 11.0      1
Name: OffenseType, dtype: int64

## Change nans with 0

In [17]:
calendar_crimes.OffenseType.replace( np.nan,0, inplace=True)

In [18]:
calendar_crimes.OffenseType.value_counts(dropna=False)

2.0     707
3.0     598
1.0     551
4.0     416
5.0     224
0.0     208
6.0     129
7.0      46
8.0      28
9.0       7
12.0      3
10.0      3
14.0      1
11.0      1
Name: OffenseType, dtype: int64

# merge calendar_crimes with score

In [19]:
merge_data = pd.merge(calendar_crimes,df,  left_index=True, right_index=True, how='left')

# change column names
#merge_data.columns = ['offenses','away_team','win']

merge_data.head()

,OffenseType,Opponent,win
days,,,
2010-01-01,5.0,NaN,NaN
2010-01-02,0.0,NaN,NaN
2010-01-03,1.0,NaN,NaN
2010-01-04,6.0,NaN,NaN
2010-01-05,1.0,NaN,NaN


In [20]:
merge_data['Opponent'].value_counts(dropna=False)

NaN                     2873
Tulsa                      4
Tulane                     4
Texas State                3
UCF                        3
SMU                        3
Rice                       2
Temple                     2
UTEP                       2
Memphis                    2
East Carolina              2
Navy                       2
Cincinnati                 1
Nevada-Las Vegas           1
South Florida              1
Lamar                      1
Vanderbilt                 1
Louisville                 1
Southern Mississippi       1
Connecticut                1
Grambling State            1
UTSA                       1
UCLA                       1
UAB                        1
Tennessee Tech             1
Georgia State              1
Marshall                   1
Mississippi State          1
Louisiana Tech             1
North Texas                1
Southern                   1
Texas Tech                 1
Name: Opponent, dtype: int64

## Cleanup colums

In [21]:
merge_data.columns  = ['crime', 'away_team', 'won']

In [22]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 3 columns):
crime        2922 non-null float64
away_team    49 non-null object
won          49 non-null float64
dtypes: float64(2), object(1)
memory usage: 171.3+ KB


In [23]:
merge_data.crime.value_counts(dropna=False)

2.0     707
3.0     598
1.0     551
4.0     416
5.0     224
0.0     208
6.0     129
7.0      46
8.0      28
9.0       7
12.0      3
10.0      3
14.0      1
11.0      1
Name: crime, dtype: int64

In [24]:
merge_data.crime = merge_data.crime.astype(int)

In [25]:
merge_data.crime.value_counts(dropna=False)

2     707
3     598
1     551
4     416
5     224
0     208
6     129
7      46
8      28
9       7
12      3
10      3
11      1
14      1
Name: crime, dtype: int64

In [26]:
merge_data

,crime,away_team,won
days,,,
2010-01-01,5,NaN,NaN
2010-01-02,0,NaN,NaN
2010-01-03,1,NaN,NaN
2010-01-04,6,NaN,NaN
2010-01-05,1,NaN,NaN
2010-01-06,2,NaN,NaN
2010-01-07,5,NaN,NaN
2010-01-08,2,NaN,NaN
2010-01-09,1,NaN,NaN


# Display non-nans

In [27]:
merge_data.dropna(subset=['away_team', 'won'])

,crime,away_team,won
days,,,
2010-09-04,2,Texas State,1.0
2010-09-10,6,UTEP,1.0
2010-09-25,5,Tulane,1.0
2010-10-09,3,Mississippi State,0.0
2010-11-05,5,UCF,0.0
2010-11-13,9,Tulsa,0.0
2011-09-03,3,UCLA,1.0
2011-09-24,4,Georgia State,1.0
2011-10-08,3,East Carolina,1.0


In [28]:
merge_data.to_csv('data/final/UH1017_final.csv')

In [29]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 3 columns):
crime        2922 non-null int64
away_team    49 non-null object
won          49 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 171.3+ KB
